In [8]:
from pgml import *
from AKV import *
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def draw_graph(model: Model):
    DG = nx.DiGraph()
    world_size = model.worlds_size
    DG.add_nodes_from(list(range(model.worlds_size)))
    DG.add_weighted_edges_from(
        [
            (i, j, model.relation[i][j])
            for i in range(world_size)
            for j in range(world_size)
            if model.relation[i][j] != 0
        ]
    )
    # labels = nx.get_edge_attributes(G, "weights")
    nx.draw(DG)

In [3]:
# Faintly connected graph...
NUM_AGENTS = 10
WEAK_INFLUENCE = 0.1
STRONG_INFLUENCE = 0.5
influence_graph = InfluenceGraphs.faintly(10, WEAK_INFLUENCE, STRONG_INFLUENCE)

# Not necessary for faintly connected graph, but it is necessary for any non symetric graph.
relation = [
    [influence_graph[j][i] for j in range(NUM_AGENTS)] for i in range(NUM_AGENTS)
]

# Remove self-loops.
for i in range(10):
    relation[i][i] = 0

# ...with mildly initial state.
belief_array = [InitialConfigurations.mildly(NUM_AGENTS)]
belief_array.append([1 - belief_array[0][i] for i in range(NUM_AGENTS)])

In [19]:
formula = Box(Variable("p"))
akv_model = AKV(belief_array, influence_graph)

# Run AKV 10 times
NUM_STEPS = 10
for i in range(NUM_STEPS):
    akv_model.overall_class_update()

# Get polarization array
# polarization = akv_model.get_polarization(k=11)

# Create a PGML model for each step
pgml_models = []
for i in range(NUM_STEPS + 1):
    pgml_models += [
        Model(
            NUM_AGENTS,
            relation,
            {"p": akv_model.states[i][0].tolist()},
            {"p": akv_model.states[i][1].tolist()},
        )
    ]

# Build table for agent
ALPHA = Box(Variable("p"))
BETA = Diamond(Variable("p"))
AGENT = 0
table = [
    {
        # "polarization": polarization[i],
        "agent valuation1": akv_model.states[i][0][AGENT],
        "agent valuation2": akv_model.states[i][1][AGENT],
        "pgml alpha valuation": ALPHA.valuation1(pgml_models[i], AGENT),
        "compare": ">"
        if ALPHA.valuation1(pgml_models[i], AGENT)
        > BETA.valuation1(pgml_models[i], AGENT)
        else "<",
        "pgml beta valuation": BETA.valuation1(pgml_models[i], AGENT),
    }
    for i in range(NUM_STEPS + 1)
]

# Dataframe
pd.DataFrame(table)

,agent valuation1,agent valuation2,pgml alpha valuation,compare,pgml beta valuation
0,0.200000,0.800000,0.240000,<,0.360000
1,0.350000,0.650000,0.370000,<,0.430000
2,0.425000,0.575000,0.435000,<,0.465000
3,0.462500,0.537500,0.467500,<,0.482500
4,0.481250,0.518750,0.483750,<,0.491250
5,0.490625,0.509375,0.491875,<,0.495625
6,0.495312,0.504687,0.495938,<,0.497812
7,0.497656,0.502344,0.497969,<,0.498906
8,0.498828,0.501172,0.498984,<,0.499453
9,0.499414,0.500586,0.499492,<,0.499727
